Copyright 2021, https://github.com/inactivitytimeout

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

# Obstacle Avoidance Example

This notebook demonstrates how to move forward, and stop when an obstacle is detected.

In [1]:
import RPi.GPIO as GPIO
from smbus2 import SMBus
import time
import threading

In [2]:
# Uncomment to silence channel already in use warning
# GPIO.setwarnings(False)

# GPIO.BCM specifiesBroadcom chip pin numbers. 
GPIO.setmode(GPIO.BCM)

GPIO_ECHO = 5   # INPUT
GPIO_TRIG = 6   # OUTPUT

In [3]:
# Set up the OUTPUT pin. This transmits a trigger pulse.
GPIO.setup(GPIO_TRIG, GPIO.OUT, initial=1)
GPIO.output(GPIO_TRIG,1)
time.sleep(0.01)
GPIO.output(GPIO_TRIG,0)
time.sleep(0.01)

In [4]:
# Set up the INPUT pin. This listens for the return "echo"
GPIO.setup(GPIO_ECHO, GPIO.IN)

In [5]:
SPEED_OF_SOUND_INCHES_PER_SEC = 1125.3 * 12.0

def get_distance():
    distance = 0.0
    ITERATION_COUNT = 1
    for i in range(ITERATION_COUNT):
        pulse_start = None
        pulse_end = None
        pulse_width = 0.0
        
        time.sleep(0.00001)
        GPIO.output(GPIO_TRIG, 1)
        time.sleep(0.00001)
        GPIO.output(GPIO_TRIG, 0)

        while GPIO.input(GPIO_ECHO) == 0:
            pulse_start = time.time()

        if pulse_start:
            while GPIO.input(GPIO_ECHO) == 1:
                pulse_end = time.time()

            pulse_width = pulse_end - pulse_start
            distance += (pulse_width / 2.0) * SPEED_OF_SOUND_INCHES_PER_SEC
            
    distance /= ITERATION_COUNT
    return distance

In [7]:
# Use /dev/i2c1
I2C_BUS = SMBus(1)

I2C_ADDRESS = 0x18
I2C_COMMAND = 0xff

# Motor
I2C_STOP     = 0x210A
I2C_FORWARD  = 0x220A
I2C_BACKWARD = 0x230A
I2C_LEFT     = 0x240A
I2C_RIGHT    = 0x250A

I2C_LEFT_SPEED_SLOW  = 0x2605
I2C_LEFT_SPEED_FAST  = 0x260A
I2C_RIGHT_SPEED_SLOW = 0x2705
I2C_RIGHT_SPEED_FAST = 0x270A

I2C_HEADLIGHT_LEFT_OFF  = 0x3600
I2C_HEADLIGHT_LEFT_ON   = 0x3601
I2C_HEADLIGHT_RIGHT_OFF = 0x3700
I2C_HEADLIGHT_RIGHT_ON  = 0x3701

In [8]:
# Initialize speed to fast before we start moving around
I2C_BUS.write_word_data(I2C_ADDRESS, I2C_COMMAND, I2C_LEFT_SPEED_FAST)
time.sleep(0.01)
I2C_BUS.write_word_data(I2C_ADDRESS, I2C_COMMAND, I2C_RIGHT_SPEED_FAST)
time.sleep(0.01)

In [9]:
# Move car forward then stop when obstacle is in the way 
# When obstacle is detected, move backward - back to the starting point

start_time = time.time()
forward_duration = 0.0
distance = get_distance()
initial_distance = distance

while distance > 6.0 and forward_duration < 10.0:
    print("forward: distance", distance)
    I2C_BUS.write_word_data(I2C_ADDRESS, I2C_COMMAND, I2C_FORWARD)
    time.sleep(0.05)
    I2C_BUS.write_word_data(I2C_ADDRESS, I2C_COMMAND, I2C_STOP)
    time.sleep(0.05)
    forward_duration = time.time() - start_time
    distance = get_distance()

start_time = time.time()
reverse_duration = 0.0
distance = get_distance()
while distance < initial_distance and reverse_duration < forward_duration:
    print("reverse: distance", distance)
    I2C_BUS.write_word_data(I2C_ADDRESS, I2C_COMMAND, I2C_BACKWARD)
    time.sleep(0.05)
    I2C_BUS.write_word_data(I2C_ADDRESS, I2C_COMMAND, I2C_STOP)
    time.sleep(0.05)
    reverse_duration = time.time() - start_time
    distance = get_distance()


forward: distance 45.63010282516479
forward: distance 45.95688300132751
forward: distance 43.931811761856075
forward: distance 43.588934040069574
forward: distance 41.254789924621576
forward: distance 39.65469388961792
forward: distance 37.78576884269714
forward: distance 36.43357501029968
forward: distance 34.315138006210326
forward: distance 31.568896722793575
forward: distance 30.366410064697263
forward: distance 28.07411956787109
forward: distance 26.717096471786498
forward: distance 24.828854370117185
forward: distance 23.39456305503845
forward: distance 21.034662866592406
forward: distance 19.329932785034178
forward: distance 17.3515444278717
forward: distance 16.121691942214966
forward: distance 14.062815856933593
forward: distance 12.192281055450438
forward: distance 10.6050630569458
forward: distance 9.87423448562622
forward: distance 7.802480363845825
forward: distance 8.332089614868163
forward: distance 9.813063812255859
reverse: distance 5.283214473724365
reverse: distance 

In [10]:
GPIO.cleanup()